In [ ]:
import csv
import json
import math
import multiprocessing
from os import path
import random

from deap import base, creator, tools
import geopandas as gpd
import matplotlib
import numpy as np
import pysal
from tqdm import tqdm, tqdm_notebook

# Force matplotlib to plot from notebook
%matplotlib inline
# Increase default plot size
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

In [ ]:
# Create a fitness class which maximises compactness and minimises 
# crime and population differences
creator.create('FitnessMulti', base.Fitness, weights=(1.0, -1.0, -1.0))

# Create individuals as simple list chromosomes
creator.create('Individual', list, fitness=creator.FitnessMulti)

# Define helper function for ingesting seed individuals
def init_individual(icls, content):
    return icls(content)

# Define helper function for ingesting seed population
def init_population(pcls, ind_init, filename):
    with open(filename, 'r') as file:
        contents = json.load(file)
    return pcls(ind_init(c) for c in contents)

# Register population_guess
toolbox = base.Toolbox()
toolbox.register('individual_guess', init_individual, creator.Individual)
toolbox.register('population_guess', init_population, list, toolbox.individual_guess, 'population.json')

In [ ]:
# Import the cleaned neighborhood data and address naming bug with .rename()
neighborhoods_with_border = gpd.read_file(
    path.join('..', 'maps', 'clean', 'neighborhoods.shp')
).rename(columns={'neighborho': 'neighborhood'})
neighborhoods = neighborhoods_with_border.query('neighborhood != "Border"').reset_index(drop=True)

# Replace the districts with NaN
neighborhoods.district = np.NaN

# Import the neighbor relationships
with open('neighbors.json') as file:
    neighbors = json.load(file)
    
# Re-set the keys as strings 
neighbors = {int(k): v for k, v in neighbors.items()}

In [ ]:
# Define helper function for plotting chromosomes
def plot_chromosome(units, individual):
    units = neighborhoods.copy()
    units['district'] = individual
    
    units.dissolve(by='district').plot()

In [ ]:
def evaluate(individual):
    units = neighborhoods.copy()
    units.district = individual
    
    # Aggregate into districts for comparison
    zones = units.dissolve(by='district', aggfunc='sum')
    
    compactness_score = evaluate_compactness(zones)
    crime_score = evaluate_count(zones.crimes)
    population_score = evaluate_count(zones.population)
    
    del units, zones
    
    return(compactness_score, crime_score, population_score)
    
    
def evaluate_count(counts):
    total = sum(counts)
    max_delta = max(counts) - min(counts)
    
    # Find p and set to 1 if extremely large
    p = max_delta / total
    if (p > 1): p = 1
    
    return p

    
def evaluate_compactness(zones):
    zones.compactness = (4 * math.pi * zones.area) / zones.length
    return min(zones.compactness)

In [ ]:
mutation_limit = 3

def pear_mutate(individual):
    
    # Copy the current individual
    current_individual = toolbox.clone(individual)
    
    # Insert the current chromosome into the geodata
    units = neighborhoods.copy()
    units.district = individual
    
    # Aggregate into districts for comparison
    zones = units.dissolve(by='district', aggfunc='sum')
    
    # Get neighbors for each zone
    zone_neighbors = pysal.weights.Rook.from_dataframe(zones).neighbors
    
    # Extract and shuffle zones
    zones_shuffled = list(zone_neighbors.keys())
    random.shuffle(zones_shuffled)
    
    # Loop through shuffled list and perform shift mutation
    for zone in zones_shuffled:
        
        # If the current zone contains only one 
        # district, skip the current shift
        if len(units[units['district'] == zone].index.values) <= mutation_limit: 
            continue
        else:
            # Randomly select a destination for shifting units
            dest_zone = random.choice(zone_neighbors[zone])
            current_individual = pear_shift(zone, dest_zone, units, current_individual)
        
        return current_individual
    
        
def pear_shift(source_zone, dest_zone, units, individual):
    
    current_individual = toolbox.clone(individual)
    
    # Select the units in the source district
    source_units = units[units['district'] == source_zone].index.values
    
    # Shuffle them for looping
    random.shuffle(source_units)
    
    # Select a starting point for the mutation inside the source zone
    sub_zone = []
    for unit in source_units:
        # Continue this loop until a border unit is found
        if len(sub_zone) > 0: break
        
        # If neighbor is in the destination zone, the current unit is
        # on the border between the two zones
        for neighbor in neighbors[unit]:
            if units.loc[neighbor]['district'] == dest_zone: 
                sub_zone.append(unit)
                break
    
    # Select three other units and impose an artificial limit to 
    # prevent infinite cycling
    iteration_count = 0
    while len(sub_zone) < mutation_limit:
        # Find neighbor units of sub_zone
        current_neighbors = []
        for unit in sub_zone:
            current_neighbors = current_neighbors + neighbors[unit]
        
        # Generate a random number between 1 and the length of the neighbor set
        number_to_subset = random.randint(1, len(current_neighbors))
        
        # Randomly choose a subset of the neighbor set 
        new_member = random.sample(current_neighbors, 1)
        
        # Join the new members to the sub_zone
        sub_zone = sub_zone + new_member
        
        # Remove repeated members
        sub_zone = list(set(sub_zone))
        
        # Break if trapped
        if iteration_count >= 10: break
    
    # Copy the current into a mutant
    mutant_individual = toolbox.clone(current_individual)
    
    # Replace the corresponding values in the mutant chromosome
    for unit in sub_zone:
        mutant_individual[unit] = dest_zone
        
    # Return If the mutant still has three districts with at least three members
    invalid = False
    for zone in [0, 1, 2]:
        if mutant_individual.count(zone) < mutation_limit: 
            invalid = True
            break
            
    if invalid:
        return current_individual
    else:
        return mutant_individual

In [ ]:
# Initialise multiprocessing pool
pool = multiprocessing.Pool()

# Import the starting population
population = toolbox.population_guess()

for gen in tqdm_notebook(range(100)):
    # Clone the entire population
    offspring = pool.map(toolbox.clone, tqdm_notebook(population, desc='Clone population'))
    
    # Delete old fitness values
    for mutant in tqdm_notebook(offspring, desc='Delete old fitness values'):
        del mutant.fitness.values
    
    # Apply mutation to offspring
    offspring = pool.map(pear_mutate, tqdm_notebook(offspring, desc='Mutate offspring'))

    # Calculate fitness
    fitnesses = map(evaluate, tqdm_notebook(offspring, desc='Evaluate fitness'))
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

#     with open('output.csv', 'a') as file:
#         csv_file = csv.writer(file)
#         for individual in tqdm_notebook(offspring, desc='Write out results'):
#             csv_file.writerow(individual + list(individual.fitness.values))
    
    population[:] = offspring
    
pool.close()